<a href="https://colab.research.google.com/github/drehnstrom/spanner-examples/blob/main/colab-notebooks/Spanner_Samples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Authenticate to Google Cloud

You need to log into Google Cloud with a valid account before running the notebook. Your account has to have permissions to create Spanner databases. 

You also need a project with Spanner enabled.

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('You are authenticated!')

## Install the Spanner Python API

You will be prompted to restart the runtime. Do so. 

In [ ]:
! pip install --upgrade google-cloud-spanner

## Set the following variables 

You at least need to change the Project ID variable to your project's ID. You can change the other variables if you like. 

In [ ]:
project_id = 'roi-spanner'
instance_id = 'my-spanner-instance'
processing_units = 100
database_id = 'pets-db'

OPERATION_TIMEOUT_SECONDS = 240

print("Done")

## Create a Spanner Instance

In [ ]:
import time
from google.cloud import spanner

def create_instance_with_processing_units(instance_id, processing_units):
    """Creates an instance."""
    spanner_client = spanner.Client(project=project_id)

    config_name = "{}/instanceConfigs/regional-us-central1".format(
        spanner_client.project_name
    )

    instance = spanner_client.instance(
        instance_id,
        configuration_name=config_name,
        display_name="spanner-instance-python",
        # Use node_count attribute to create instance using nodes
        # instead of Processing Units
        # node_count = 1,
        processing_units=processing_units,
        labels={
            "created": str(int(time.time())),
        },
    )

    # create() returns a long-running operation
    operation = instance.create()

    print("Waiting for operation to complete...")
    operation.result(OPERATION_TIMEOUT_SECONDS)

    print(
        "Created instance {} with {} processing units".format(
            instance_id, instance.processing_units
        )
    )


# Call the function
create_instance_with_processing_units(instance_id, processing_units)

## Create the Pets Database

In [ ]:
def create_database(instance_id, database_id):
    """Creates a database and tables for sample data."""
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)

    database = instance.database(
        database_id,
        ddl_statements=[
            """CREATE TABLE Owners (
                  OwnerID STRING(36) NOT NULL,
                  OwnerName STRING(MAX) NOT NULL
               ) PRIMARY KEY (OwnerID)""",
            """CREATE TABLE Pets (
                  PetID STRING(36) NOT NULL, 
                  OwnerID STRING(36) NOT NULL, 
                  PetType STRING(MAX) NOT NULL,
                  PetName STRING(MAX) NOT NULL,
                  Breed STRING(MAX) NOT NULL,
                  CONSTRAINT FK_OwnerPet FOREIGN KEY (OwnerID) REFERENCES Owners (OwnerID),
              ) PRIMARY KEY (PetID)""",
            """CREATE INDEX PetsByOwnerID ON Pets(OwnerID)""",
            """CREATE INDEX PetsByType ON Pets(PetType)""",
            """CREATE INDEX OwnersByName ON Owners(OwnerName)""",
        ],
    )

    # create() returns a long-running operation
    operation = database.create()

    print("Waiting for operation to complete...")
    operation.result(OPERATION_TIMEOUT_SECONDS)

    print("Created database {} on instance {}".format(database_id, instance_id))

create_database(instance_id, database_id)


## This just show the Data Definition code for the Pets database 

In [ ]:
def get_database_ddl(instance_id, database_id):
    """Gets the database DDL statements."""
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)
    ddl = spanner_client.database_admin_api.get_database_ddl(database=database.name)

    print("Retrieved database DDL for {}".format(database_id))

    for statement in ddl.statements:
        print(statement)


get_database_ddl(instance_id, database_id)

## Add a few test records

In [ ]:
import uuid

def insert_data(instance_id, database_id):
    doug_id = str(uuid.uuid4())
    john_id = str(uuid.uuid4())
    sue_id = str(uuid.uuid4())

    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)

    with database.batch() as batch:
        batch.insert(
            table="Owners",
            columns=("OwnerID", "OwnerName"),
            values=[
                (doug_id, u"Doug"),
                (john_id, u"John"),
                (sue_id, u"Sue"),
            ],
        )

        batch.insert(
            table="Pets",
            columns=("PetID", "OwnerID", "PetType", "PetName", "Breed"),
            values=[
                (str(uuid.uuid4()), doug_id, u"Dog", u"Noir", u"Schnoodle"),
                (str(uuid.uuid4()), doug_id, u"Dog", u"Bree", u"Mutt"),
                (str(uuid.uuid4()), doug_id, u"Cat", u"Tom", u"Alley"),
                (str(uuid.uuid4()), john_id, u"Dog", u"Duke", u"GoldenDoodle"),\
                (str(uuid.uuid4()), john_id, u"Dog", u"Sparky", u"Poodle"),
                (str(uuid.uuid4()), john_id, u"Turtle", u"Cuff", u"Box"),
                (str(uuid.uuid4()), john_id, u"Turtle", u"Link", u"Box"),
                (str(uuid.uuid4()), sue_id, u"Cat", u"Cleo", u"Domestic"),
            ],
        )

    print("Inserted data.")


insert_data(instance_id, database_id)

## Run a query using SQL

In [ ]:
def run_simple_query(sql):
  # Instantiate a client.
  spanner_client = spanner.Client(project=project_id)

  # Get a Cloud Spanner instance by ID.
  instance = spanner_client.instance(instance_id)

  # Get a Cloud Spanner database by ID.
  database = instance.database(database_id)

  # Execute a simple SQL statement.
  with database.snapshot() as snapshot:
      results = snapshot.execute_sql(sql)
      for row in results:
        print(row)


sql = """SELECT OwnerName, PetName, PetType, Breed 
         FROM Owners 
         JOIN Pets ON Owners.OwnerID = Pets.OwnerID;"""


run_simple_query(sql)

## Read data from a table

In [ ]:
def read_data(instance_id, database_id):
    """Reads sample data from the database."""
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)

    with database.snapshot() as snapshot:
        keyset = spanner.KeySet(all_=True)
        results = snapshot.read(
            table="Pets", columns=("PetID", "PetName", "PetType", "Breed"), keyset=keyset
        )

        for row in results:
            print(u"PetID: {}, PetName: {}, PetType: {}, Breed: {}".format(*row))


# Call the function
read_data(instance_id, database_id)

## Stale Read

Reads sample data from the database. The data is exactly 15 seconds stale.

In [ ]:
def read_stale_data(instance_id, database_id):

    import datetime

    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)

    # Everything is the same as the prior example except staleness
    staleness = datetime.timedelta(seconds=15)
    with database.snapshot(exact_staleness=staleness) as snapshot:
        keyset = spanner.KeySet(all_=True)
        results = snapshot.read(
            table="Pets", columns=("PetID", "PetName", "PetType", "Breed"), keyset=keyset,
        )

        for row in results:
            print(u"PetID: {}, PetName: {}, PetType: {}, Breed: {}".format(*row))

# Call the function
read_stale_data(instance_id, database_id)


## Run Query using Index with Parameters

In [ ]:
def query_data_with_index(instance_id, database_id, owner_name="Doug"):

    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)

    params = {"owner_name": owner_name}
    param_types = {
        "owner_name": spanner.param_types.STRING,
    }

    with database.snapshot() as snapshot:
        results = snapshot.execute_sql(
            """SELECT OwnerName, PetName, PetType, Breed 
               FROM Owners 
               JOIN Pets ON Owners.OwnerID = Pets.OwnerID 
               WHERE OwnerName = @owner_name"""
            "",
            params=params,
            param_types=param_types,
        )

        for row in results:
            print(row)

query_data_with_index(instance_id, database_id, "John")

## Run Query using Index

In [ ]:
def get_pets_by_type(instance_id, database_id, pet_type="Dog"):

    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)

    params = {"pet_type": pet_type}
    param_types = {
        "pet_type": spanner.param_types.STRING,
    }

    with database.snapshot() as snapshot:
        results = snapshot.execute_sql(
            """SELECT *
               FROM Pets@{FORCE_INDEX=PetsByType} 
               WHERE PetType = @pet_type"""
            "",
            params=params,
            param_types=param_types,
        )

        for row in results:
            print(row)

get_pets_by_type(instance_id, database_id, pet_type="Dog")

## Read data with an Index

In [ ]:
def read_owners_with_index(instance_id, database_id):
    
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)

    with database.snapshot() as snapshot:
        keyset = spanner.KeySet(all_=True)
        results = snapshot.read(
            table="Owners", 
            columns=("OwnerName",), 
            keyset=keyset,
            index="OwnersByName",
        )

        for row in results:
          print(row)


read_owners_with_index(instance_id, database_id)

## Add an Index

In [ ]:
def add_index(instance_id, database_id):
    """Adds a simple index to the example database."""
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)

    operation = database.update_ddl(
        ["CREATE INDEX PetsByTypeBreedName ON Pets(PetType,Breed,PetName)"]
    )

    print("Waiting for operation to complete...")
    operation.result(OPERATION_TIMEOUT_SECONDS)

    print("Added the PetsByTypeBreedName index.")

# Call the function
add_index(instance_id, database_id)

## Read with Index just Created


In [ ]:
def read_pets_with_index(instance_id, database_id):
    
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)

    with database.snapshot() as snapshot:
        keyset = spanner.KeySet(all_=True)
        results = snapshot.read(
            table="Pets", 
            columns=("PetType","Breed","PetName"), 
            keyset=keyset,
            index="PetsByTypeBreedName",
        )

        for row in results:
          print(row)

# Call the function
read_pets_with_index(instance_id, database_id)

## Insert some Rows

In [ ]:
import uuid

def insert_owner(instance_id, database_id, owner_id, owner_name):
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)

    def insert_owners(transaction, owner_id, owner_name):
        params = {"owner_id": owner_id, 
                  "owner_name": owner_name,
                  }
        param_types = {
          "owner_id": spanner.param_types.STRING,
          "owner_name": spanner.param_types.STRING,
          } 

        row_ct = transaction.execute_update(
            """INSERT Owners (OwnerID, OwnerName)
               VALUES (@owner_id, @owner_name)""",
               params=params,
               param_types=param_types,
        )
        print("{} record(s) inserted.".format(row_ct))

    database.run_in_transaction(insert_owners, owner_id, owner_name)



# Call the function
insert_owner(instance_id, database_id, str(uuid.uuid4()), "Dave" )


## Run Update Query with Partitioned DML

In [ ]:
def update_data_with_dml(instance_id, database_id):
    
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)

    row_ct = database.execute_partitioned_dml(
    """UPDATE Pets SET PetType = 'Canine'
       WHERE PetType = 'Dog'""")
    
    print("{} record(s) updated.".format(row_ct))


# Call the function
update_data_with_dml(instance_id, database_id)

## Check Changes

In [ ]:
def run_query(sql):
  spanner_client = spanner.Client(project=project_id)
  instance = spanner_client.instance(instance_id)
  database = instance.database(database_id)

  # Execute a simple SQL statement.
  with database.snapshot() as snapshot:
      results = snapshot.execute_sql(sql)
      for row in results:
        print(row)


sql = """SELECT OwnerName, PetName, PetType, Breed 
         FROM Owners 
         LEFT JOIN Pets ON Owners.OwnerID = Pets.OwnerID;"""

run_query(sql)

## Run Read-Only Transaction

In [ ]:
def read_only_transaction(instance_id, database_id):
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)

    # Both reads use the same snapshot, so are executed at the same timestamp
    with database.snapshot(multi_use=True) as snapshot:
        # First Read.
        results = snapshot.execute_sql("SELECT OwnerName FROM Owners")

        print("Results from first read:")
        for row in results:
            print(row)

        #Second Read
        results = snapshot.execute_sql("SELECT PetName, PetType, Breed FROM Pets")

        print("Results from second read:")
        for row in results:
            print(row)


read_only_transaction(instance_id, database_id)

## Delete All the Data

In [26]:
def delete_data_with_dml(instance_id, database_id):
    
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)

    def delete_pets_owners(transaction):
        row_ct = transaction.execute_update(
            "DELETE FROM Pets WHERE true = true"
        )
        row_ct += transaction.execute_update(
            "DELETE FROM Owners WHERE true = true"
        )

        print("{} record(s) deleted.".format(row_ct))

    database.run_in_transaction(delete_pets_owners)

# Call the function
delete_data_with_dml(instance_id, database_id)


13 record(s) deleted.


## Delete the Database

In [27]:
def delete_database(instance_id, database_id):
    
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    database = instance.database(database_id)
    
    database.drop()

    print("{} database dropped".format(database_id))

    

# Call the function
delete_database(instance_id, database_id)

pets-db database dropped


## Delete the Instance

In [28]:
def delete_spanner_instance(instance_id):
    
    spanner_client = spanner.Client(project=project_id)
    instance = spanner_client.instance(instance_id)
    instance.delete()

    print("{} instance deleted".format(instance_id))

# Call the function
delete_spanner_instance(instance_id)

my-spanner-instance instance deleted


In [29]:
%%bash

gcloud config set project roi-spanner

gcloud spanner instances list


NAME          DISPLAY_NAME  CONFIG                NODE_COUNT  PROCESSING_UNITS  STATE  INSTANCE_TYPE
spannerdbsrv  spannerdbsrv  regional-us-central1              100               READY  PROVISIONED


Updated property [core/project].
